<a href="https://colab.research.google.com/github/nanabonsu92/Deep-Learning_TIES4911/blob/main/TIES4911_Demo02_Bonsu_Afrane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.	Build a Neural Network based Classification model for the following dataset (http://users.jyu.fi/~olkhriye/ties4911/demos/demo1/Automobile_price_data_Raw_set.csv)
2.  Take column [make] as dependent parameter (our classification target) and following columns as independent parameters: body-style, wheel-base, engine-size, horsepower, peak-rpm, highway-mpg, price. Predict the car brand for the following inputs:

( body-style, wheel-base, engine-size, horsepower, peak-rpm, highway-mpg, price )

[ 'sedan', 103.5, 164, 121, 4250, 25, 24565]

[ 'hatchback', 86.6, 92, 58, 4800, 54, 6479]


In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#loading the dataset
dataframe = pd.read_csv("/content/gdrive/MyDrive/Deep Learning 2024/Task 1/Automobile_price_data_Raw_set.csv")

In [4]:
dataframe

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,95.0,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114.0,5400.0,23,28,16845.0
201,-1,95.0,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160.0,5300.0,19,25,19045.0
202,-1,95.0,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134.0,5500.0,18,23,21485.0
203,-1,95.0,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106.0,4800.0,26,27,22470.0


In [5]:
# Dropping rows with any missing values (NaNs)
df = dataframe.dropna()

In [6]:
df

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0
6,1,158.0,audi,gas,std,four,sedan,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110.0,5500.0,19,25,17710.0
8,1,158.0,audi,gas,turbo,four,sedan,fwd,front,105.8,...,131,mpfi,3.13,3.40,8.3,140.0,5500.0,17,20,23875.0
10,2,192.0,bmw,gas,std,two,sedan,rwd,front,101.2,...,108,mpfi,3.50,2.80,8.8,101.0,5800.0,23,29,16430.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,95.0,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114.0,5400.0,23,28,16845.0
201,-1,95.0,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160.0,5300.0,19,25,19045.0
202,-1,95.0,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134.0,5500.0,18,23,21485.0
203,-1,95.0,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106.0,4800.0,26,27,22470.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 3 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          153 non-null    int64  
 1   normalized-losses  153 non-null    float64
 2   make               153 non-null    object 
 3   fuel-type          153 non-null    object 
 4   aspiration         153 non-null    object 
 5   num-of-doors       153 non-null    object 
 6   body-style         153 non-null    object 
 7   drive-wheels       153 non-null    object 
 8   engine-location    153 non-null    object 
 9   wheel-base         153 non-null    float64
 10  length             153 non-null    float64
 11  width              153 non-null    float64
 12  height             153 non-null    float64
 13  curb-weight        153 non-null    int64  
 14  engine-type        153 non-null    object 
 15  num-of-cylinders   153 non-null    object 
 16  engine-size        153 non

In [8]:
df.describe()

,symboling,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
count,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000
mean,0.738562,121.156863,98.316993,172.579739,65.622876,53.905882,2462.222222,119.575163,3.303137,3.238693,10.113856,96.509804,5117.320261,26.424837,31.986928,11478.830065
std,1.207276,35.945521,5.183359,11.537413,1.928611,2.239603,475.193136,30.301054,0.266642,0.294405,3.854074,30.851964,467.261066,6.156610,6.499987,5780.915957
min,-2.000000,65.000000,86.600000,141.100000,60.300000,49.400000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,15.000000,18.000000,5118.000000
25%,0.000000,94.000000,94.500000,166.300000,64.000000,52.500000,2094.000000,97.000000,3.080000,3.110000,8.700000,70.000000,4800.000000,23.000000,28.000000,7499.000000
50%,1.000000,113.000000,97.000000,172.400000,65.400000,54.100000,2365.000000,110.000000,3.270000,3.270000,9.000000,90.000000,5200.000000,26.000000,32.000000,9279.000000
75%,2.000000,148.000000,101.200000,177.800000,66.500000,55.500000,2808.000000,136.000000,3.540000,3.410000,9.400000,114.000000,5500.000000,31.000000,37.000000,15040.000000
max,3.000000,256.000000,115.600000,202.600000,71.700000,59.800000,4066.000000,258.000000,3.940000,4.170000,23.000000,200.000000,6600.000000,49.000000,54.000000,35056.000000


In [9]:
# Adjusting feature and target columns for classification
feature_columns = ['body-style', 'wheel-base', 'engine-size', 'horsepower', 'peak-rpm', 'highway-mpg', 'price']
target_column = 'make'  # Changing target to 'make'

In [10]:
# Extracting features and target
X = df[feature_columns]
Y = df[target_column]

In [11]:
# Encoding categorical data
X_encoded = pd.get_dummies(X, columns=['body-style'])

In [12]:
# Encoding target variable
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)
Y_encoded = tf.keras.utils.to_categorical(Y_encoded)  # One-hot encoding for target

In [13]:
# Splitting the data
x_train, x_test, y_train, y_test = train_test_split(X_encoded, Y_encoded, test_size=0.2, random_state=0)

In [14]:
# Initializing MinMaxScaler
scaler = MinMaxScaler()

# Scaling features
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [21]:
# Building the Neural Network model for classification
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(x_train_scaled.shape[1],)),
    tf.keras.layers.Dropout(0.2),  # Adding dropout for regularization
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),  # Another dropout layer
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(Y_encoded.shape[1], activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),  # Adjusting learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
# Training the model
model.fit(x_train_scaled, y_train, epochs=500, batch_size=100)

Epoch 1/500
2/2 [==============================] - 1s 12ms/step - loss: 2.8987 - accuracy: 0.0656
Epoch 2/500
2/2 [==============================] - 0s 10ms/step - loss: 2.8209 - accuracy: 0.0574
Epoch 3/500
2/2 [==============================] - 0s 10ms/step - loss: 2.7150 - accuracy: 0.1639
Epoch 4/500
2/2 [==============================] - 0s 8ms/step - loss: 2.6218 - accuracy: 0.1803
Epoch 5/500
2/2 [==============================] - 0s 8ms/step - loss: 2.5462 - accuracy: 0.2213
Epoch 6/500
2/2 [==============================] - 0s 8ms/step - loss: 2.5317 - accuracy: 0.2213
Epoch 7/500
2/2 [==============================] - 0s 7ms/step - loss: 2.4590 - accuracy: 0.2131
Epoch 8/500
2/2 [==============================] - 0s 7ms/step - loss: 2.4069 - accuracy: 0.2377
Epoch 9/500
2/2 [==============================] - 0s 10ms/step - loss: 2.3567 - accuracy: 0.2295
Epoch 10/500
2/2 [==============================] - 0s 7ms/step - loss: 2.3514 - accuracy: 0.2131
Epoch 11/500
2/2 [=======

In [23]:
# Evaluating the model
loss, accuracy = model.evaluate(x_test_scaled, y_test)
print("Accuracy of the trained model: ", accuracy)


1/1 [==============================] - 0s 139ms/step - loss: 3.2132 - accuracy: 0.6774
Accuracy of the trained model:  0.6774193644523621


In [24]:
# Preparing the new input data for prediction
new_inputs = np.array([
    ['sedan', 103.5, 164, 121, 4250, 25, 24565],
    ['hatchback', 86.6, 92, 58, 4800, 54, 6479]
])

In [25]:
# Correcting the DataFrame creation by including all necessary columns
feature_columns_for_new_input = ['body-style', 'wheel-base', 'engine-size', 'horsepower', 'peak-rpm', 'highway-mpg', 'price']
new_inputs_df = pd.DataFrame(new_inputs, columns=feature_columns_for_new_input)

new_inputs_encoded = pd.get_dummies(new_inputs_df, columns=['body-style'])

new_inputs_encoded = new_inputs_encoded.reindex(columns=x_train.columns, fill_value=0)

In [26]:
# Scaling the new input data
new_inputs_scaled = scaler.transform(new_inputs_encoded)

In [27]:
# Predicting with the trained model
predictions = model.predict(new_inputs_scaled)
predicted_labels = np.argmax(predictions, axis=1)
predicted_brands = label_encoder.inverse_transform(predicted_labels)

print("Predicted car brands: ", predicted_brands)

1/1 [==============================] - 0s 94ms/step
Predicted car brands:  ['bmw' 'honda']
